### Composite AGN Cutout Query Creator
This notebook will create `SQL` query requests for a series of cutout images of a specific single quasar to create training data for the transfer-learning phase on real data for `DualFinder`

In [1]:
import matplotlib.pyplot as plt
from matplotlib import colors, cm
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import glob
import scipy
import csv
import pandas as pd
import random
import matplotlib.image as mpimg
from astropy.table import Table
from tqdm.notebook import tqdm,trange
import requests
import time
import tarfile
from schwimmbad import MultiPool
import multiprocess as mp
from astropy.io import fits
import matplotlib as mpl
import os
from os.path import exists
import shutil
import requests
from offset_AGN_dataset_creator import DatasetCreator
import sys
sys.path.append("data_preprocessing/")
sys.path.append("data_preprocessing/training_datasets/")
from FITSViewer import display_images_with_buttons
#from dual_AGN_utils import crop_center
%load_ext autoreload
%autoreload 2


In [2]:
print(os.getcwd())
#os.makedirs("Rotated_AGN_diff_fltrs/HSC_R/")
#os.makedirs("Rotated_AGN_diff_fltrs/HSC_G/")
#os.makedirs("Rotated_AGN_diff_fltrs/HSC_Z/")
#os.makedirs("Rotated_AGN_diff_fltrs/HSC_Y/")

/vast/palmer/scratch/urry/iam37/Project_DRAGON/DRAGON_CNN


In [3]:
def crop_center(img, cropx, cropy):
    
    #Function from 
    #https://stackoverflow.com/questions/39382412/crop-center-portion-of-a-numpy-image
    
    y, x, *_ = img.shape
    startx = x // 2 - (cropx // 2)
    #print(startx)
    starty = y // 2 - (cropy // 2) 
    #print(starty)
    return img[starty:starty + cropy, startx:startx + cropx, ...]

In [4]:
def imgdwnldr_gen(args):
    
    df,num_start,num_stop,fltr = args
    
    t = Table()
    t["rerun"] = ["pdr3_wide"]*len(df[num_start:num_stop])
    t["filter"] = [fltr]*len(df[num_start:num_stop])
    #t["tract"] = df["tract"][num_start:num_stop]
    t["ra"] = df["ra"][num_start:num_stop]
    t["dec"] = df["dec"][num_start:num_stop]
    t["type"] = ["coadd"]*len(df[num_start:num_stop])
    t["sh"] = ["16asec"]*len(df[num_start:num_stop])
    t["sw"] = ["16asec"]*len(df[num_start:num_stop])
    t["name"] = df["name"][num_start:num_stop]
                  
    return t

In [5]:
def table_writer(args):
    
    table, i, filepath_prefix, overwrite, tableformat, comment= args
    #os.chdir(filepath_prefix)
    #print("CURRENT WORKING DIRECTORY: " + str(os.getcwd()))
    table.write(filepath_prefix + "download_sql" + ".txt",
                overwrite=overwrite,format=tableformat,
                comment=comment)
    #os.chdir("/Users/moskowitzi/Library/CloudStorage/Dropbox/First_Year_at_Yale/Summer_2022/Quasar_Research")
#args = (test2DF, 0, 35, "HSC-R")
#secondTestTable = imgdwnldr_gen(args)
#filepath_prefix = 'input_second_test_HSC_new'
def write_dwnldr_files(df,step,filepath_prefix = None,write=True,
                      overwrite=True, fltr = None):   
                  
    len_df = len(df)
    iters = int(len_df/step)
    #print(len_df)
    lwr_ends = range(0,len_df,step)
    upr_ends = range(step,len_df+step,step)
    fltrs = [fltr]*(iters+1)
    dfs = [df]*(iters+1)
    
    args_imgdwnldr_gen = list(zip(dfs,lwr_ends,upr_ends,fltrs))
    #args_imgdwnldr_gen = (dfs,lwr_ends,upr_ends,fltrs)
    #imgdwnldr_gen(args_imgdwnldr_gen)
    with mp.Pool() as pool:
        tables = list(tqdm(pool.imap(imgdwnldr_gen,args_imgdwnldr_gen), total = iters+1))
    
    if write is True:
        paths = [filepath_prefix]*(iters+1)
        overwrites = [overwrite]*(iters+1)
        tableformats = ['ascii.commented_header']*(iters+1)
        comments = ["#?"]*(iters+1)
        i_s = np.arange(iters+1)
    
        args_table_writer = list(zip(tables,i_s,paths,overwrites,tableformats,comments))
        #args_table_writer = (tables,i_s,paths,overwrites,tableformats,comments)
        #table_writer(args_table_writer)
        
        with mp.Pool() as pool:
            exit_codes = list(tqdm(pool.imap(table_writer,args_table_writer), total = iters+1))
        #print("got here after table writer")
        
    else:
        for table in tables:
            print(table)
    return tables
def write_rotation_test_coordinates(quasarName, separation, raStart, decStart, fltr):
    count = 0
    degree_separation = separation/3600
    raList = []
    decList = []
    quasarNameList = []
    random_angles = np.random.uniform(0.0, 359.9, 200)
    for i in random_angles:
        ra = raStart + degree_separation* np.cos(i)
        dec = decStart + degree_separation* np.sin(i)
        raList.append(ra)
        decList.append(dec)
        quasarNameNew = quasarName + "_HSC_" + str(fltr) + "_rotated_" + str(count) + "_degrees"
        #print(quasarNameNew)
        quasarNameList.append(quasarNameNew)
        count+=10
    return raList, decList, quasarNameList
#Automatic downloaders for downloading from HSC servers:
def cutout_requester(args):
    
    session, listpath_prefix, write_path_prefix, i = args
    print(f"listpath_prefix: {listpath_prefix}")
    print(f"write_path_prefix: {write_path_prefix}")
    
    files = {
        'list': (listpath_prefix + "download_sql.txt", open(listpath_prefix + "download_sql.txt", 'rb')),
    }
    
    with session.post('https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout', 
                      files=files, auth=('moskowitzi', 'UCiNwD3CHzbprC8G1QM2aUxpQIZdp2cOjyQWOO7J'), stream=True) as response:
        
        total_size = int(response.headers.get('content-length', 0))
        chunk_size = len(i)  # Define chunk size
        
        with open(write_path_prefix + "download.tar", 'wb') as f, tqdm(
            total=total_size, unit='B', unit_scale=True, desc=write_path_prefix + "download.tar") as progress_bar:
            
            for chunk in response.iter_content(chunk_size):
                if chunk:  # Filter out keep-alive new chunks
                    f.write(chunk)
                    progress_bar.update(len(chunk))
def members(tf):
    
    for i, member in enumerate(tf.getmembers()):
        if i == 0:
            l = len(member.path.split("/")[0]) + 1
        
        member.path = member.path[l:]
        yield member
def write_band_download_cutouts(quasarNameList, raList, decList, fltr, filepath, filepath_prefix, separation):
    
    header = ['ra', 'dec', 'name']
    print(len(raList))
    #if not (exists(filepath) or exists(filepath_prefix)):
        #os.makedirs(filepath)
        #os.makedirs(filepath_prefix)
    if not exists(filepath):
        os.makedirs(filepath)
    if not exists(filepath_prefix):
        os.makedirs(filepath_prefix)
    with open(filepath + "catalog_" + str(separation) + ".csv", 'w', encoding = 'UTF8') as f: 
        writer = csv.writer(f)
        writer.writerow(header)
        for j in range(len(raList)):
            #print("raList[j]: " + str(raList[j]))
            #print(type(raList[j]))
            combinedData = [raList[j], decList[j], quasarNameList[j]]
            writer.writerow(combinedData)
    #print("got here")
    test_df = pd.read_csv(filepath + "catalog_" + str(separation) + ".csv", delimiter = ',')
    write = True
    overwrite = True
    #print(test_df)
    write_dwnldr_files(test_df, fltr = fltr, step=len(raList), write=write,overwrite=overwrite,
                filepath_prefix = filepath_prefix)
        
#Function to actuall extract all the tarballs
def extractor(args):
    tar_file_prefix, write_path, i = args
    
    with tarfile.open(tar_file_prefix + f"download.tar") as tf:
        tf.extractall(write_path,members=members(tf))

In [6]:
def reducedAmplitudeTestFile(quasarName, percentage, separation, raStart, decStart, fltr, filepath = None):
    quasarName = quasarName
    
    #raStart = 154.60814662744912
    #decStart = 0.34943320872888634
    raList, decList, quasarNameList = write_rotation_test_coordinates(quasarName, separation, raStart, decStart, fltr)
    #print("got here")
    header = ['ra', 'dec', 'name']
#args = secondTestTable, filepath_prefix, overwrite, tableformat, comment
#table_writer(args)
    #print(len(quasarNameList))
    #print(len(raList))
    #print(len(decList))
    if not exists(filepath):
        os.makedirs(filepath)
    with open(filepath + "rotation_test_reduced_amplitude_"+str(separation)+".csv", 'w', encoding = 'UTF8') as f: 
        writer = csv.writer(f)
        writer.writerow(header)
        for j in range(len(raList)):
            combinedData = [raList[j], decList[j], quasarNameList[j]]
            writer.writerow(combinedData)
    #print("got here")
    test_df = pd.read_csv(filepath + "rotation_test_reduced_amplitude_"+str(separation)+".csv", delimiter = ',')
    write = True
    overwrite = True
    if not exists(filepath + "downloader_files/downloader_for_" + str((separation)) + "/"):
        os.makedirs(filepath + "downloader_files/downloader_for_" + str((separation)) + "/")
    tables = write_dwnldr_files(test_df,step=len(raList), fltr = fltr, write=write,overwrite=overwrite,
                filepath_prefix= filepath + "downloader_files/downloader_for_" + str((separation)) + "/")
    #print(test_df)

In [7]:
img = fits.getdata("cutout-HSC-G-9226-pdr3_wide-240530-152449.fits")
plt.imshow(img, vmin = np.percentile(img, 1), vmax = np.percentile(img, 99))

FileNotFoundError: [Errno 2] No such file or directory: 'cutout-HSC-G-9226-pdr3_wide-240530-152449.fits'

In [22]:
#os.chdir("/vast/palmer/scratch/urry/iam37/")
#All Spring Equitorial Quasars in SDSS and HSC
starttime = time.time()
fits_table_dr16q = '../DR16Q_v4.fits'
hdu1 = fits.open(fits_table_dr16q)
data = hdu1[1].data
min_row_fall_1 = 668635
max_row_fall_1 = 750414
min_row_fall_2 = 1
max_row_fall_2 = 116727
quasarNameList_fall = []
quasarRAList_fall = []
quasarDecList_fall = []
quasarZList_fall = []
"""Fall Equatorial Band"""
for j in tqdm(range(min_row_fall_1, max_row_fall_2)):
    #if(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and (data[j].field(6) == "QSO" or data[j].field(7) == "QSO" or data[j].field(6) == data[j].field(7) == "UNK")):
    if(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and (data[j].field(6) == "QSO" or data[j].field(7) == "QSO" or data[j].field(6) == data[j].field(7) == "UNK") and data[j].field(28)):
    
        quasarName = data[j].field(6) + " " + data[j].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        #quasarNameNoSpaceNoPeriod = quasarNameNoSpace.replace(".", "_")
        #quasarNameNoPlus = quasarNameNoSpaceNoPeriod.replace("+", "_")
        inputQuasarName = quasarNameNoSpace
        quasarNameList_fall.append(inputQuasarName)
        quasarRAList_fall.append(data[j].field(1))
        quasarDecList_fall.append(data[j].field(2))
        quasarZList_fall.append(data[j].field(28))
for j in tqdm(range(min_row_fall_2, max_row_fall_2)):
    if(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and (data[j].field(6) == "QSO" or data[j].field(7) == "QSO" or data[j].field(6) == data[j].field(7) == "UNK") and data[j].field(28)<=1.59):
        quasarName = data[j].field(6) + " " + data[j].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        inputQuasarName = quasarNameNoSpace
        quasarNameList_fall.append(inputQuasarName)
        quasarRAList_fall.append(data[j].field(1))
        quasarDecList_fall.append(data[j].field(2))
        quasarZList_fall.append(data[j].field(28))
quasarNameList_fall = np.asarray(quasarNameList_fall)
#for j in range(len(quasarNameList_fall)):
    #if quasarNameList_fall[j].find("QSO") == -1:
        #print(quasarNameList_fall[j])
        #print(j)
print(len(quasarNameList_fall))
print(len(quasarDecList_fall))
header = ['ra', 'dec', 'Quasar name', 'Z']
#print(len(quasarNameList))
if not exists("HSC_survey_bands/"):
    os.makedirs("HSC_survey_bands/")
with open("HSC_survey_bands/fall_equitorial_quasar_list.csv", 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0, len(quasarNameList_fall)):
        combinedData = [quasarRAList_fall[i], quasarDecList_fall[i], quasarNameList_fall[i], quasarZList_fall[i]]
        writer.writerow(combinedData)
fall_df = pd.read_csv("HSC_survey_bands/fall_equitorial_quasar_list.csv", delimiter=',')
#print(fall_df)
"""Spring Equatorial Band"""
valid_rows1 = data[170000:552181]
print(data[0][0])
print(data[0].field(2))
quasarNameList_spring = []
quasarRAList = []
quasarDecList = []
quasarZList = []
#We know that HSC's field contains RA = 130º at 199012, I will now test smaller RA's to see if they are also contained within the 
#survey field
for i in tqdm(range(191617, 552181)):
    if (data[i].field(2)>= -7.0 and data[i].field(2) <= 1.0 and (data[i].field(6) == "QSO" or data[i].field(7) == "QSO" or data[i].field(6) == data[i].field(7) == "UNK") and data[i].field(28)<=1.59):
        quasarName = data[i].field(6) + " " + data[i].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        inputQuasarName = quasarNameNoSpace
        quasarNameList_spring.append(inputQuasarName)
        quasarRAList.append(data[i].field(1))
        quasarDecList.append(data[i].field(2))
        quasarZList.append(data[i].field(28))
quasarNameList_spring = np.asarray(quasarNameList_spring)
#for j in range(len(quasarNameList_spring)):
    #if quasarNameList_spring[j].find("QSO") == -1:
        #print(quasarNameList_spring[j])
        #print("Not a QSO Object")
        #print(j)
#header = ['ra', 'dec', 'Quasar name']
print(len(quasarNameList_spring))
with open("HSC_survey_bands/spring_equitorial_quasar_list.csv", 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0, len(quasarNameList_spring)):
        combinedData = [quasarRAList[i], quasarDecList[i], quasarNameList_spring[i], quasarZList[i]]
        writer.writerow(combinedData)
spring_df = pd.read_csv("HSC_survey_bands/spring_equitorial_quasar_list.csv", delimiter=',')
#print(spring_df)
#for i in range(0, len(spring_df)):
    #numpyVersion = spring_df.loc[i].to_numpy()
    #print(numpyVersion)
    #if (numpyVersion[1] <=-6.0):
        #print("Something went wrong here")
        #print(spring_df.loc[i])
#display(spring_df.to_string())
"""
with open("spring_equitorial_quasar_list_hsc_download.csv", "w", encoding='UTF8') as p:
    writer = csv.writer(p)
    header = ["#?", "rerun", "filter", "ra", "dec", "sw", "sh", "name"]
    writer.writerow(header)
    for i in range(0, len(quasarNameList)):
        combinedData = [NaN, "pdr3_wide", "HSC-R", quasarRAList[i], quasarDecList[i], "0.01asec", "0.01asec", "spring_equitorial_"+str(i)]
        writer.writerow(combinedData)
"""
#print(quasarNameList)
#print(quasarZList)
#In this section of the download, we are working in the Spring Equitorial survey path, which goes from roughly
#120 degrees to 225 degrees of RA and -7 to 1 degrees of Dec
"""Northern Sky Band"""
min_row_north = 448834
max_row_north = 602111
quasarNameList_north = []
quasarRAList_north = []
quasarDecList_north = []
quasarZList_north = []
for k in tqdm(range(min_row_north, max_row_north)):
    if(data[k].field(2) >= 42.5 and data[k].field(2) <= 44.0 and (data[k].field(6) == "QSO" or data[k].field(7) == "QSO" or data[k].field(6) == data[k].field(7) == "UNK") and data[k].field(28)<=1.59):
        quasarName = data[k].field(6) + " " + data[k].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        inputQuasarName = quasarNameNoSpace
        quasarNameList_north.append(inputQuasarName)
        quasarRAList_north.append(data[k].field(1))
        quasarDecList_north.append(data[k].field(2))
        quasarZList_north.append(data[k].field(28))
quasarNameList_north = np.asarray(quasarNameList_north)
#for k in range(len(quasarNameList_north)):
    #if quasarNameList_north[k].find("QSO") == -1:
        #print(quasarNameList_north[k])
        #print(k)
print(len(quasarNameList_north))
print("Total number of QSO objects = " + str(len(quasarNameList_fall) + len(quasarNameList_spring) + len(quasarNameList_north)))
with open("HSC_survey_bands/northern_sky_quasar_list.csv", 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0, len(quasarNameList_north)):
        combinedData = [quasarRAList_north[i], quasarDecList_north[i], quasarNameList_north[i], quasarZList_north[i]]
        writer.writerow(combinedData)
northern_df = pd.read_csv("HSC_survey_bands/northern_sky_quasar_list.csv", delimiter=',')
endtime = time.time()
print("--- %s seconds ---" % (endtime - starttime))

0it [00:00, ?it/s]

  0%|          | 0/116726 [00:00<?, ?it/s]

15555
15555
000000.15+353104.2
35.51784115297793


  0%|          | 0/360564 [00:00<?, ?it/s]

4474


  0%|          | 0/153277 [00:00<?, ?it/s]

2854
Total number of QSO objects = 22883
--- 26.727097034454346 seconds ---


In [ ]:
# Finds 'STAR' objects in the SDSS catalogue instead of just QSO objects.
start_time = time.time()
fits_table_dr16q = '../DR16Q_v4.fits'
hdu1 = fits.open(fits_table_dr16q)
data = hdu1[1].data
# Define constants
min_row_fall_1 = 668635
max_row_fall_1 = 750414
min_row_fall_2 = 1
max_row_fall_2 = 116727
header = ['ra', 'dec', 'Object name', 'Z']
# Function to process data
def process_data(data, min_row, max_row, ra_range, dec_range, filename, object_type='STAR'):
    object_name_list = []
    ra_list = []
    dec_list = []
    z_list = []

    for j in tqdm(range(min_row, max_row)):
        if ra_range[0] <= data[j].field(2) <= ra_range[1] and (data[j].field(6) == object_type or data[j].field(7) == object_type or data[j].field(6) == data[j].field(7) == object_type):
            object_name = data[j].field(6) + " " + data[j].field(0)
            object_name_no_space = object_name.replace(" ", "_")
            object_name_list.append(object_name_no_space)
            ra_list.append(data[j].field(1))
            dec_list.append(data[j].field(2))
            z_list.append(data[j].field(28))

    # Save to CSV
    if not os.path.exists("HSC_survey_bands/"):
        os.makedirs("HSC_survey_bands/")

    with open(f"HSC_survey_bands/{filename}", 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i in range(len(object_name_list)):
            combined_data = [ra_list[i], dec_list[i], object_name_list[i], z_list[i]]
            writer.writerow(combined_data)

    return len(object_name_list)

# Process Fall Equatorial Band
fall_count = process_data(data, min_row_fall_1, max_row_fall_1, (-1.0, 7.0), (-1.0, 7.0), 'fall_equitorial_object_list.csv', 'STAR')
fall_count += process_data(data, min_row_fall_2, max_row_fall_2, (-1.0, 7.0), (-1.0, 7.0), 'fall_equitorial_object_list.csv', 'STAR')

# Process Spring Equatorial Band
spring_count = process_data(data, 191617, 552181, (-7.0, 1.0), (-7.0, 1.0), 'spring_equitorial_object_list.csv', 'STAR')

# Process Northern Sky Band
north_count = process_data(data, 448834, 602111, (42.5, 44.0), (42.5, 44.0), 'northern_sky_object_list.csv', 'STAR')

fall_df = pd.read_csv("HSC_survey_bands/fall_equitorial_object_list.csv", delimiter=',')
spring_df = pd.read_csv("HSC_survey_bands/spring_equitorial_object_list.csv", delimiter=',')
northern_df = pd.read_csv("HSC_survey_bands/northern_sky_object_list.csv", delimiter=',')

# Print results
print(f"Fall count: {fall_count}")
print(f"Spring count: {spring_count}")
print(f"Northern count: {north_count}")
print(f"Total number of STAR objects: {fall_count + spring_count + north_count}")

# End timer
endtime = time.time()
print("--- %s seconds ---" % (endtime - start_time))

In [11]:
print(spring_df.head(20))
print(fall_df.head(20))
print(northern_df.head(20))

            ra       dec             Quasar name         Z
0   128.101873  0.640967  QSO_083224.44+003827.4  3.148698
1   128.119200 -4.890228  UNK_083228.60-045324.8 -1.000000
2   128.175790  0.745218  QSO_083242.18+004442.7  2.938249
3   128.180986 -0.110710  QSO_083243.43-000638.5  2.910548
4   128.211311  0.456109  QSO_083250.71+002721.9  2.505143
5   128.213605  0.271332  QSO_083251.26+001616.7  2.643090
6   128.216005  0.732931  QSO_083251.84+004358.5  2.420306
7   128.232129  0.786146  QSO_083255.71+004710.1  2.933359
8   128.245663  0.762376  QSO_083258.95+004544.5  2.875120
9   128.259624 -0.052714  QSO_083302.30-000309.7  2.472723
10  128.300261 -0.145306  QSO_083312.06-000843.1  2.583532
11  128.312892  0.760944  QSO_083315.09+004539.3  2.643800
12  128.357256  0.738579  QSO_083325.74+004418.8  2.400242
13  128.369081  0.959607  QSO_083328.57+005734.5  2.278242
14  128.374453  0.004155  QSO_083329.86+000014.9  2.441090
15  128.437107  0.019690  QSO_083344.90+000110.8  2.5309

In [33]:
percentages = np.arange(0.1, 1.0, 0.1)
separations_distribution_1 = np.random.uniform(0.55, 2.5, 200)
#angle = np.random.uniform(0.0, 359.0, 200)
#quasar_name_1 = "QSO_000035.59-003146.1"
#quasar_name_2 = "QSO_131227.61-002134.3"
#ra_start_1 = 0.14833262219804055 + 0.67 * np.cos(0)/3600.0
#dec_start_1 = -0.5294901735587073
#ra_start_2 = 198.11505695798235 + 0.67 * np.cos(0)/3600.0
#dec_start_2 = -0.3595408479420831

rotation_AGN_candidate_images = []
rotation_AGN_candidate_names = []
rotation_ra_list = []
rotation_dec_list = []
for ii, confirmed_single in tqdm(enumerate(glob.glob("data_preprocessing/training_datasets/single_AGN_datasets/confirmed_single_z<1.59/**/*.fits", recursive=True))):
    try:
        with fits.open(confirmed_single, memmap = False) as hdul:
            img = hdul[1].data
            img = crop_center(img, 94, 94)
            rotation_AGN_candidate_images.append(img)
            name = confirmed_single.split('/')[-1].split('.fits')[0]
            rotation_AGN_candidate_names.append(name)
            location = northern_df[northern_df['Quasar name'] == name]
            if not location.empty:
                ra = location.iloc[0]['ra']
                dec = location.iloc[0]['dec']
                rotation_ra_list.append(ra)
                rotation_dec_list.append(dec)
            else:
                print("ERROR: cannot find quasar in northern sky dataframe")
    except OSError:
        print(f"file {confirmed_single} seems to be corrupted or cannot be opened, moving on...")

#filepath = "Rotated_AGN_diff_fltrs/HSC_I/"
filepath = "Randomized_Rotated_AGN_diff_fltrs/HSC_G/"

fltr = 'HSC-G'
for percentage in tqdm(percentages):
    for angular_separation in tqdm(separations_distribution_1):
        angular_separation = np.round(angular_separation, 2)
        
        rand_index = np.random.choice(len(rotation_AGN_candidate_names))
        rand_quasar_name = rotation_AGN_candidate_names[rand_index]
        rand_quasar_ra_start = rotation_ra_list[rand_index] + 0.67 * np.cos(0)/3600.0
        rand_quasar_dec_start = rotation_dec_list[rand_index]
        percentages = random.uniform(0.1, 1.0)
        tables = reducedAmplitudeTestFile(rand_quasar_name, percentage, angular_separation, rand_quasar_ra_start, rand_quasar_dec_start, fltr, filepath = filepath)
        #if rand_quasar_name == quasar_name_1:
            #tables = reducedAmplitudeTestFile(rand_quasar_name, percentage, angular_separation, ra_start_1, dec_start_1, fltr, filepath = filepath)
        #else:
            #tables = reducedAmplitudeTestFile(rand_quasar_name, percentage, angular_separation, ra_start_2, dec_start_2, fltr, filepath = filepath)


0it [00:00, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#This script creates new 16asec x 16asec cutouts of QSO 101825.95+002057.9 to create dual AGN images that are upscaled to the correct size for the rest of the dataset
percentages = np.arange(0.1, 1.0, 0.1)
separations = np.arange(0.55, 2.5, 0.05)
angle = np.arange(0.0, 359.0, 1.0)
quasar_name = "QSO_101823.39-012400.6"
ra_start = 154.59748850760818 + 0.55 * np.cos(0)/3600.0
dec_start = -1.4001730205213287
filepath = "dual_AGN_training_data/Rotated_AGN_diff_fltrs/HSC_I/"
fltr = 'HSC-I'
for ii, percentage in tqdm(enumerate(percentages)):
    for j, angular_separation in enumerate(separations):
        angular_separation = np.round(angular_separation, 2)
        tables = reducedAmplitudeTestFile(quasar_name, percentage, angular_separation, ra_start, dec_start, fltr, filepath = filepath)


In [ ]:
#This script will create cutout CSV and .txt files for rotated STAR objects. These objects will be stars chosen for their point-source resemblance
# and their lack of background noise. These sources will be rotated around central AGN candidates to resemble a chance star-quasar alignment.
star_name = "STAR_105335.60-002157.8"
start_ra = 163.398361 + 0.55*np.cos(0)/3600.0
start_dec = -0.366077
star_filepath = "Rotated_STAR_files/HSC_I/"
fltr = 'HSC-I'
percentages = np.arange(0.1, 1.0, 0.1)
separations = np.arange(0.55, 2.5, 0.05)
for ii, percentage in tqdm(enumerate(percentages)):
    for j, angular_separation in enumerate(separations):
        angular_separation = np.round(angular_separation, 2)
        tables = reducedAmplitudeTestFile(star_name, percentage, angular_separation, start_ra, start_dec, fltr, filepath = star_filepath)

In [15]:
def members(tarf):
    
    for i, member in enumerate(tarf.getmembers()):
        if i == 0:
            l = len(member.path.split("/")[0]) + 1
        
        member.path = member.path[l:]
        yield member
        
        
#Function to actuall extract all the tarballs
def extractor(args):
    tar_file_prefix, write_path, i = args
    
    with tarfile.open(tar_file_prefix + "download.tar") as tarf:
        tarf.extractall(write_path,members=members(tarf))

In [ ]:
extract = True
separations = np.arange(1.95, 2.5, 0.05)
for j, angular_separation in tqdm(enumerate(separations)):
    total_files = 1404

    i_s = np.arange(total_files)
    prefix = "Rotated_STAR_files/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/downloaded_images"
    dwnldr_txt_filepath = "Rotated_STAR_files/" + str("HSC_I")+ "/downloader_files/downloader_for_" + str(np.round(angular_separation, 2))+"/"
    write_path = "Rotated_STAR_files/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/opened_images"

    #if extract is True:
        #with mp.Pool() as p:
            #exit_codes = list(tqdm(p.imap_unordered(cutout_requester,requester_args), total=len(i_s)))
    total_threads = 2
    #url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout/'
    url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout'
    if not exists("Rotated_STAR_files/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/"):
        os.makedirs("Rotated_STAR_files/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/")
    with requests.Session() as session:
        args_cutout_requester = (session,dwnldr_txt_filepath,write_path,i_s)
        cutout_requester(args_cutout_requester)
    extractor_args = (write_path,prefix,i_s)
    extractor(extractor_args)

In [ ]:
extract = True
print(separations_distribution_1)
for j, angular_separation in tqdm(enumerate(separations_distribution_1[47:])):
    total_files = 1404

    i_s = np.arange(total_files)
    #prefix = "Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/downloaded_images"
    prefix = "Randomized_Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/downloaded_images"
    #dwnldr_txt_filepath = "Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloader_files/downloader_for_" + str(np.round(angular_separation, 2))+"/"
    dwnldr_txt_filepath = "Randomized_Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloader_files/downloader_for_" + str(np.round(angular_separation, 2))+"/"
    
    #write_path = "Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/opened_images"
    write_path = "Randomized_Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/opened_images"

    #if extract is True:
        #with mp.Pool() as p:
            #exit_codes = list(tqdm(p.imap_unordered(cutout_requester,requester_args), total=len(i_s)))
    total_threads = 2
    #url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout/'
    url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout'
    if not exists("Randomized_Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/"):
        os.makedirs("Randomized_Rotated_AGN_diff_fltrs/" + str("HSC_G")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/")
    #with MultiPool(processes=2) as pool:
        #with requests.Session() as session: #just to keep things clearn
            #sessions = [session]*len(i_s)
            #args_cutout_requester = list(zip(sessions,dwnldr_txt_filepath,write_path,i_s))
            #exit_codes = list(tqdm(pool.imap_unordered(cutout_requester,args_cutout_requester), total = len(i_s)))
    with requests.Session() as session:
        args_cutout_requester = (session,dwnldr_txt_filepath,write_path,i_s)
        cutout_requester(args_cutout_requester)
    extractor_args = (write_path,prefix,i_s)
    extractor(extractor_args)
    #with open("Rotated_AGN_diff_fltrs/" + str(fltr)+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/" + "download.tar", 'wb') as out_file:

    #extractor_args = list(zip(prefix,write_path,i_s))
    #with mp.Pool() as p:
        #exit_codes = list(tqdm(p.imap_unordered(extractor,extractor_args), total=len(i_s)))

In [ ]:
extract = True
for j, angular_separation in tqdm(enumerate(separations)):
    total_files = 1404

    i_s = np.arange(total_files)
    prefix = "dual_AGN_training_data/Rotated_AGN_diff_fltrs/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/downloaded_images"
    dwnldr_txt_filepath = "dual_AGN_training_data/Rotated_AGN_diff_fltrs/" + str("HSC_I")+ "/downloader_files/downloader_for_" + str(np.round(angular_separation, 2))+"/"
    write_path = "dual_AGN_training_data/Rotated_AGN_diff_fltrs/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/opened_images"

    #if extract is True:
        #with mp.Pool() as p:
            #exit_codes = list(tqdm(p.imap_unordered(cutout_requester,requester_args), total=len(i_s)))
    total_threads = 2
    #url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout/'
    url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout'
    if not exists("dual_AGN_training_data/Rotated_AGN_diff_fltrs/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/"):
        os.makedirs("dual_AGN_training_data/Rotated_AGN_diff_fltrs/" + str("HSC_I")+ "/downloaded_images/" + str(np.round(angular_separation, 2))+"_arcsecond_separations/")
    with requests.Session() as session:
        args_cutout_requester = (session,dwnldr_txt_filepath,write_path,i_s)
        cutout_requester(args_cutout_requester)
    extractor_args = (write_path,prefix,i_s)
    extractor(extractor_args)

In [15]:
bands = ["HSC-G", "HSC-R", "HSC-Z", "HSC-Y", "HSC-I"]
for fltr in tqdm(bands):
    print(fltr)
    total_files = 100000
    north_filepath_prefix = "entire_bands/" + str(fltr) + "/northern_sky/"
    fall_filepath_prefix = "entire_bands/" + str(fltr) + "/fall_equatorial/"
    spring_filepath_prefix = "entire_bands/" + str(fltr) + "/spring_equatorial/"
    i_s = np.arange(total_files)
    
    north_prefix = north_filepath_prefix + "downloaded_images/"
    north_dwnldr_txt_filepath = north_filepath_prefix
    north_write_path = north_filepath_prefix + "downloaded_images/opened_images"
    
    spring_prefix = spring_filepath_prefix + "downloaded_images/"
    spring_dwnldr_txt_filepath = spring_filepath_prefix
    spring_write_path = spring_filepath_prefix + "downloaded_images/opened_images"
    
    fall_prefix = fall_filepath_prefix + "downloaded_images/"
    fall_dwnldr_txt_filepath = fall_filepath_prefix
    fall_write_path = fall_filepath_prefix + "downloaded_images/opened_images"
    
    """if not exists(fall_prefix) and not exists(fall_dwnldr_txt_filepath) and not exists(fall_write_path):
        os.makedirs(fall_prefix)
        os.makedirs(fall_dwnldr_txt_filepath)
        os.makedirs(fall_write_path)
    if not exists(spring_prefix) and not exists(spring_dwnldr_txt_filepath) and not exists(spring_write_path):
        os.makedirs(spring_prefix)
        os.makedirs(spring_dwnldr_txt_filepath)
        os.makedirs(spring_write_path)
    if not exists(north_prefix) and not exists(north_dwnldr_txt_filepath) and not exists(north_write_path):
        os.makedirs(north_prefix)
        os.makedirs(north_dwnldr_txt_filepath)
        os.makedirs(north_write_path)"""
    write_band_download_cutouts(quasarNameList_fall, quasarRAList_fall, quasarDecList_fall, fltr, fall_dwnldr_txt_filepath, fall_filepath_prefix, 0.0)
    write_band_download_cutouts(quasarNameList_spring, quasarRAList, quasarDecList, fltr, spring_dwnldr_txt_filepath, spring_filepath_prefix, 0.0)
    write_band_download_cutouts(quasarNameList_north, quasarRAList_north, quasarDecList_north, fltr, north_dwnldr_txt_filepath, north_filepath_prefix, 0.0)

    total_threads = 2
    url = 'https://hsc-release.mtk.nao.ac.jp/das_cutout/pdr3/cgi-bin/cutout/'
    with requests.Session() as session:
        fall_args_cutout_requester = (session,fall_dwnldr_txt_filepath,fall_prefix,i_s)
        spring_args_cutout_requester = (session,spring_dwnldr_txt_filepath,spring_prefix,i_s)
        north_args_cutout_requester = (session,north_dwnldr_txt_filepath,north_prefix,i_s)
        print("Fall cutouts")
        cutout_requester(fall_args_cutout_requester)
        fall_extractor_args = (fall_write_path,fall_prefix,i_s)
        extractor(fall_extractor_args)
        
        print("Spring cutouts")
        cutout_requester(spring_args_cutout_requester)
        spring_extractor_args = (spring_write_path,spring_prefix,i_s)
        extractor(spring_extractor_args)
        
        print("North cutouts")
        cutout_requester(north_args_cutout_requester)
        spring_extractor_args = (north_write_path,north_prefix,i_s)
        extractor(north_extractor_args)
    

  0%|          | 0/5 [00:00<?, ?it/s]

HSC-G
15555


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

4474


...

2854


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Fall cutouts
listpath_prefix: entire_bands/HSC-G/fall_equatorial/
write_path_prefix: entire_bands/HSC-G/fall_equatorial/downloaded_images/


FileNotFoundError: [Errno 2] No such file or directory: 'entire_bands/HSC-G/fall_equatorial/downloaded_images/download.tar'

In [ ]:
!pwd

In [ ]:
single_AGN_HSC_I_filepath = "DRAGON_CNN/data_preprocessing/training_datasets/single_AGN_datasets/HSC-I/confirmed_single_AGN/"
single_HSCI = []
single_HSCI_filenames = []
for image in tqdm(glob.glob(single_AGN_HSC_I_filepath+"*.fits")):
    single_HSCI_filenames.append(image)
    with fits.open(image) as hdul:
        img = hdul[1].data
        img = crop_center(img, 94, 94)
        single_HSCI.append(img)
display_images_with_buttons(single_HSCI, single_HSCI_filenames)

Create randomized dual AGN images using the `DatasetCreator` object and API. 

In [7]:
#dataset_creation_session = DatasetCreator()
#single_AGN_filepath_HSC_G = "data_preprocessing/training_datasets/single_AGN_datasets/confirmed_single_AGN/confirmed_single_z<1.59/"
single_AGN_filepath_HSC_G = "data_preprocessing/training_datasets/single_AGN_datasets/confirmed_single_z<1.59/"
random_dual_AGN_filepath_HSC_G = "Randomized_Rotated_AGN_diff_fltrs/HSC_G/downloaded_images/"
fits_filepath = "data_preprocessing/training_datasets/dual_AGN_datasets/new_train_data/"
if not exists(fits_filepath):
    os.makedirs(fits_filepath)
single_AGN_images = []
for image_file in tqdm(glob.glob(single_AGN_filepath_HSC_G+"**/*.fits", recursive = True)):
    with fits.open(image_file, memmap=False) as hdul:
        img = hdul[1].data
        img = crop_center(img, 94, 94)
        single_AGN_images.append(img)
randomized_dual_images = []
for image_file in tqdm(glob.glob(random_dual_AGN_filepath_HSC_G+"**/*.fits", recursive=True)):
    try:
        with fits.open(image_file, memmap=False) as hdul:
            img = hdul[1].data
            img = crop_center(img, 94, 94)
            randomized_dual_images.append(img)
    except OSError:
        print(f"file {image_file} is corrupted or otherwise cannot be opened, moving on...")
# Now create convolution of images
#for ii, single_image in tqdm(enumerate(single_AGN_images)):
for j, random_dual in tqdm(enumerate(randomized_dual_images)):
    ii = np.random.choice(len(single_AGN_images))
    #single_image = np.random.choice(single_AGN_images)
    single_image = single_AGN_images[ii]
    convolved_image = single_image + random_dual
    hdu = fits.PrimaryHDU(convolved_image)
    hdul = fits.HDUList([hdu])
    hdul.writeto(f"{fits_filepath}object{ii}_with_AGN_{j}_randomized.fits" , overwrite=True)

  0%|          | 0/2469 [00:00<?, ?it/s]

  0%|          | 0/18000 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
dataset_creation_session = DatasetCreator()
single_AGN_filepath_HSC_I = "data_preprocessing/training_datasets/single_AGN_datasets/HSC-I/confirmed_single_AGN/"
rotated_star_filepath_HSC_I = "Rotated_STAR_files/HSC_I/downloaded_images/"
fits_star_filepath_HSC_I = "data_preprocessing/training_datasets/stellar_dataset/HSC-I/train_data/"
dataset_creation_session.extract_single_galaxies(single_AGN_filepath_HSC_I)
dataset_creation_session.extract_rotated_AGN(rotated_star_filepath_HSC_I)
dataset_creation_session.create_convolution(fits_filepath = fits_star_filepath_HSC_I)

In [ ]:
dataset_creation_session = DatasetCreator()
singleton_galaxy_HSC_I = "data_preprocessing/training_datasets/singleton_galaxies/HSC-I/"
rotated_AGN_galaxy_HSC_I = "Rotated_AGN_diff_fltrs/HSC_I/downloaded_images/"
fits_offset_filepath_HSC_I = "data_preprocessing/training_datasets/offset_AGN_datasets/HSC-I/train_data/"
dataset_creation_session.extract_single_galaxies(singleton_galaxy_HSC_I)
dataset_creation_session.extract_rotated_AGN(rotated_AGN_galaxy_HSC_I)
dataset_creation_session.create_convolution(fits_filepath = fits_offset_filepath_HSC_I)

In [8]:

!pwd
dataset_creation_session = DatasetCreator()
single_AGN_filepath = "data_preprocessing/training_datasets/single_AGN_datasets/confirmed_single_AGN/"
#single_AGN_filepath = "data_preprocessing/training_datasets/single_AGN_datasets/HSC-I/confirmed_single_AGN/"
rotated_AGN_filepath = "dual_AGN_training_data/Rotated_AGN_diff_fltrs/HSC_G/downloaded_images/"
#rotated_AGN_filepath = "dual_AGN_training_data/Rotated_AGN_diff_fltrs/HSC_I/downloaded_images/"
fits_dual_AGN_filepath = "data_preprocessing/training_datastes/dual_AGN_datasets/new_train_data/"
#fits_dual_AGN_filepath = "data_preprocessing/training_datasets/dual_AGN_datasets/HSC-I/train_data/"
dataset_creation_session.extract_single_galaxies(single_AGN_filepath)
dataset_creation_session.extract_rotated_AGN(rotated_AGN_filepath)
#dataset_creation_session.extract_single_point_sources(
dataset_creation_session.create_convolution(fits_filepath = fits_dual_AGN_filepath)

/vast/palmer/scratch/urry/iam37/Project_DRAGON/DRAGON_CNN


2958it [00:11, 260.79it/s]
100%|██████████| 40/40 [00:00<00:00, 15391.94it/s]


WRITING TO FILEPATH: data_preprocessing/training_datastes/dual_AGN_datasets/new_train_data/


2958it [00:00, 3482108.12it/s]


In [ ]:
#Creates offset AGN images with downloaded confirmed non-merger galaxies. 
dataset_creation_session = DatasetCreator()
#single_AGN_filepath = "data_preprocessing/training_datasets/single_AGN_datasets/confirmed_single_AGN/"
single_galaxy_filepath = "../Rotated_AGN_diff_fltrs/HSC_G/filtered_ds/"
rotated_AGN_filepath = "data_preprocessing/training_datasets/dual_AGN_datasets/Rotated_AGN_diff_fltrs/HSC_G/downloaded_images/"
fits_dual_AGN_filepath = "data_preprocessing/training_datasets/offset_AGN_datasets/train_data/"
dataset_creation_session.extract_single_galaxies(single_galaxy_filepath)
dataset_creation_session.extract_rotated_AGN(rotated_AGN_filepath)
#dataset_creation_session.extract_single_point_sources(
dataset_creation_session.create_convolution(fits_dual_AGN_filepath)

In [ ]:
!pwd

In [ ]:
offset_training_images = []
for ii, image in tqdm(enumerate(glob.glob("data_preprocessing/training_datasets/offset_AGN_datasets/HSC-I/train_data/*.fits"))):
    try:
        with fits.open(image, memmap = False) as hdul:
            img = hdul[0].data
            if np.shape(img) != (94, 94):
                print(f"ERROR: image has shape {np.shape(img)}")
            offset_training_images.append(img)
        #print(f" {ii} images were successfully loaded, {image} is corrupted/could not be loaded")
    except OSError:
        print(f" {ii} images were successfully loaded, {image} is corrupted/could not be loaded")
        continue
random_offset_selection = np.random.choice(len(offset_training_images), size = 25)
fig, axes = plt.subplots(5, 5, figsize = (20,20))
counter = 0
for row in range(5):
    for col in range(5):
        img = offset_training_images[random_offset_selection[counter]]
        axes[row][col].imshow(img, vmin = np.percentile(img, 1), vmax = np.percentile(img, 99))
        counter += 1
plt.tight_layout()
plt.show()

In [ ]:
dual_training_images = []
for ii, image in tqdm(enumerate(glob.glob("data_preprocessing/training_datasets/dual_AGN_datasets/train_data/*.fits"))):
    try:
        with fits.open(image, memmap = False) as hdul:
            img = hdul[0].data
            if np.shape(img) != (94, 94):
                print(f"ERROR: image has shape {np.shape(img)}")
            dual_training_images.append(img)
    except OSError:
        print(f"{image} cannot be opened")
        #print(f" {ii} images were successfully loaded, {image} is corrupted/could not be loaded")
    #except OSError:
random_dual_selection = np.random.choice(len(dual_training_images), size = 25)
fig, axes = plt.subplots(5, 5, figsize = (20,20))
counter = 0
for row in range(5):
    for col in range(5):
        img = dual_training_images[random_dual_selection[counter]]
        axes[row][col].imshow(img, vmin = np.percentile(img, 1), vmax = np.percentile(img, 99))
        counter += 1
plt.tight_layout()
plt.show()